# Slack

This notebook walks through connecting LangChain to your `Slack` account.

To use this toolkit, you will need to get a token explained in the [Slack API docs](https://api.slack.com/tutorials/tracks/getting-a-token). Once you've received a SLACK_USER_TOKEN, you can input it as an environmental variable below.

In [8]:
%pip install --upgrade --quiet  slack_sdk > /dev/null
%pip install --upgrade --quiet  beautifulsoup4 > /dev/null # This is optional but is useful for parsing HTML messages
%pip install --upgrade --quiet  python-dotenv > /dev/null # This is for loading environmental variables from a .env file


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Set Environmental Variables

The toolkit will read the SLACK_USER_TOKEN environmental variable to authenticate the user so you need to set them here. You will also need to set your OPENAI_API_KEY to use the agent later.

In [9]:
# Set environmental variables here
# In this example, you set environmental variables by loading a .env file.
import dotenv

dotenv.load_dotenv()

True

## Create the Toolkit and Get Tools

To start, you need to create the toolkit, so you can access its tools later.

In [1]:
from langchain_community.agent_toolkits import SlackToolkit

toolkit = SlackToolkit()
tools = toolkit.get_tools()
tools

[SlackGetChannel(client=<slack_sdk.web.client.WebClient object at 0x10891b950>),
 SlackGetMessage(client=<slack_sdk.web.client.WebClient object at 0x10891b990>),
 SlackScheduleMessage(client=<slack_sdk.web.client.WebClient object at 0x108930fd0>),
 SlackSendMessage(client=<slack_sdk.web.client.WebClient object at 0x1089311d0>)]

## Use within an OpenAI Tools Agent

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-4")
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    prompt=prompt,
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [4]:
agent_executor.invoke(
    {
        "input": "Send a greeting to my coworkers in the #general channel. Note use `channel` as key of channel id, and `message` as key of content to sent in the channel."
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_channelid_name_dict` with `{}`


[{"id": "C05L9P6HA8N", "name": "随机", "created": 1691248496, "num_members": 1}, {"id": "C05LDF54S21", "name": "general", "created": 1691248563, "num_members": 2}, {"id": "C05M6441QU8", "name": "全体", "created": 1691248496, "num_members": 1}]
Invoking: `send_message` with `{'message': "Hello everyone, hope you're having a great day!", 'channel': 'C05LDF54S21'}`


Message sent: {'ok': True, 'channel': 'C05LDF54S21', 'ts': '1710854616.151859', 'message': {'user': 'U06QASVJT6D', 'type': 'message', 'ts': '1710854616.151859', 'bot_id': 'B06PV9X1ANR', 'app_id': 'A06PY7L4FMG', 'text': "Hello everyone, hope you're having a great day!", 'team': 'T05LDF3K01K', 'bot_profile': {'id': 'B06PV9X1ANR', 'app_id': 'A06PY7L4FMG', 'name': 'my-powerful-app', 'icons': {'image_36': 'https://a.slack-edge.com/80588/img/plugins/app/bot_36.png', 'image_48': 'https://a.slack-edge.com/80588/img/plugins/app/bot_48.png', 'image_72'

{'input': 'Send a greeting to my coworkers in the #general channel. Note use `channel` as key of channel id, and `message` as key of content to sent in the channel.',
 'output': 'I have sent a greeting message to your coworkers in the #general channel.'}

In [5]:
agent_executor.invoke(
    {
        "input": "How many channels are in the workspace? Please list out their names."
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_channelid_name_dict` with `{}`


[{"id": "C05L9P6HA8N", "name": "随机", "created": 1691248496, "num_members": 1}, {"id": "C05LDF54S21", "name": "general", "created": 1691248563, "num_members": 2}, {"id": "C05M6441QU8", "name": "全体", "created": 1691248496, "num_members": 1}]There are 3 channels in the workspace. Their names are:

1. 随机
2. general
3. 全体

> Finished chain.


{'input': 'How many channels are in the workspace? Please list out their names.',
 'output': 'There are 3 channels in the workspace. Their names are:\n\n1. 随机\n2. general\n3. 全体'}

In [6]:
agent_executor.invoke(
    {
        "input": "Tell me the number of messages sent in the #introductions channel from the past month."
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_channelid_name_dict` with `{}`


[{"id": "C05L9P6HA8N", "name": "随机", "created": 1691248496, "num_members": 1}, {"id": "C05LDF54S21", "name": "general", "created": 1691248563, "num_members": 2}, {"id": "C05M6441QU8", "name": "全体", "created": 1691248496, "num_members": 1}]I'm sorry, but I couldn't find a channel named #introductions. The available channels are #随机, #general, and #全体.

> Finished chain.


{'input': 'Tell me the number of messages sent in the #introductions channel from the past month.',
 'output': "I'm sorry, but I couldn't find a channel named #introductions. The available channels are #随机, #general, and #全体."}